# **1. Import Library**

In [2]:
import librosa as lb
import os
import subprocess
import json
import numpy as np

# **2. Prerocressing dan Fungsi Librosa**

## Lokasi File

In [3]:
video_input = "../source/Vid/interview_question_1.webm"
audio_output = "../source/Audio/interview_question_1.wav"
result_output = "../source/librosa_result/interview_question_1.json"

## Video ke Audio

In [4]:
def extract_audio(video_path, audio_path):
    # pastikan folder tujuan ada
    audio_folder = os.path.dirname(audio_path)
    os.makedirs(audio_folder, exist_ok=True)

    cmd = [
        "ffmpeg",
        "-y",               # overwrite file
        "-i", video_path,   # input video
        "-vn",              # no video
        "-acodec", "pcm_s16le",  # format WAV
        "-ar", "44100",     # sample rate
        "-ac", "1",         # mono
        audio_path
    ]

    subprocess.run(cmd, stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL)

    print(">> Audio extracted to:", audio_path)


## Fungsi Librosa

In [5]:
def librosa_processing(audio_path) :
    y, sr = lb.load(audio_path, sr=None)
    tempo, beat_frames = lb.beat.beat_track(y=y, sr=sr)
    beat_times = lb.frames_to_time(beat_frames, sr=sr)

    # Silence (jeda)
    intervals = lb.effects.split(y, top_db=30)

    return {
        "tempo": tempo,
        "beats": beat_times,
        "segments": [(s/sr, e/sr) for s, e in intervals]
    }

## Testing Fungsi

In [6]:
extract_audio(video_input,audio_output)

>> Audio extracted to: ../source/Audio/interview_question_1.wav


In [7]:
result = librosa_processing(audio_output)

# **3. Simpan Hasil**

## Fungsi to Object

In [8]:
def to_python_type(obj):
    if isinstance(obj, np.ndarray):
        return obj.tolist()
    if isinstance(obj, (np.float32, np.float64)):
        return float(obj)
    if isinstance(obj, (np.int32, np.int64)):
        return int(obj)
    if isinstance(obj, dict):
        return {k: to_python_type(v) for k, v in obj.items()}
    if isinstance(obj, list):
        return [to_python_type(item) for item in obj]
    return obj

## Fungsi Save Librosa Result

In [9]:
def save_librosa_result(result_path,result) :
    clean_result = to_python_type(result)
    with open(result_path, "w") as f:
        json.dump(clean_result, f, indent=4)

## Fungsi Load Json Librosa Result

In [10]:
def load_librosa_result(result_path) :
    with open(result_path, "r") as f:
        saved_result = json.load(f)
    return saved_result

## Menggunakan Fungsi

In [15]:
save_librosa_result(result_output,result)
saved_result = load_librosa_result(result_output)
print(saved_result)


{'tempo': [132.51201923076923], 'beats': [0.22058956916099773, 0.7546485260770975, 1.18421768707483, 1.6137868480725623, 1.9969160997732427, 2.4380952380952383, 2.8212244897959184, 3.320453514739229, 3.7848526077097504, 4.260861678004535, 4.702040816326531, 5.154829931972789, 5.607619047619048, 6.060408163265306, 6.478367346938776, 6.884716553287982, 7.291065759637188, 7.685804988662132, 8.068934240362811, 8.428843537414966, 8.858412698412698, 9.287981859410431, 9.729160997732427, 10.170340136054422, 10.62312925170068, 11.029478458049887, 11.459047619047618, 11.900226757369614, 12.34140589569161, 12.759365079365079, 13.177324263038548, 13.653333333333334, 14.071292517006803, 14.605351473922903, 15.12780045351474, 15.603809523809524, 16.068208616780044, 16.532607709750568, 17.00861678004535, 17.41496598639456, 17.867755102040817, 18.274104308390022, 18.68045351473923, 19.098412698412698, 19.51637188208617, 20.01560090702948, 20.44517006802721, 20.874739229024943, 21.292698412698414, 21.

# **4. Analisa Non Verbal**

## Fungsi Normalisasi Json

In [12]:
def normalize_result(result):

    if isinstance(result["tempo"], list):
        result["tempo"] = float(result["tempo"][0])
    else:
        result["tempo"] = float(result["tempo"])

    result["beats"] = [float(b) for b in result["beats"]]

    fixed_segments = []
    for seg in result["segments"]:
        start, end = seg
        fixed_segments.append((float(start), float(end)))

    result["segments"] = fixed_segments

    return result

## Membuat fungsi analisa non-verbal

In [13]:
def analyze_nonverbal(raw_result):
    result = normalize_result(raw_result)
    tempo = result["tempo"]
    beats = result["beats"]
    segments = result["segments"]

    # Hitung jumlah dan durasi jeda
    pause_count = len(segments)
    pause_durations = [e - s for s, e in segments]
    avg_pause = sum(pause_durations) / len(pause_durations) if pause_durations else 0

    analysis = {}

    # Tempo analysis
    if tempo > 130:
        analysis["tempo_interpretation"] = "Cepat — kemungkinan gugup atau terlalu bersemangat."
    elif tempo < 80:
        analysis["tempo_interpretation"] = "Lambat — mungkin hati-hati atau sedang berpikir."
    else:
        analysis["tempo_interpretation"] = "Normal — ritme bicara stabil dan percaya diri."

    # Pause analysis
    if pause_count > 10:
        analysis["pause_interpretation"] = "Banyak jeda — bisa berarti gugup atau belum siap."
    elif avg_pause > 2:
        analysis["pause_interpretation"] = "Jeda panjang — Anda berpikir sebelum menjawab."
    else:
        analysis["pause_interpretation"] = "Jeda natural — alur bicara lancar."

    # Beat pattern
    if len(beats) > 0:
        analysis["rhythm_interpretation"] = "Ritme bicara cukup konsisten."
    else:
        analysis["rhythm_interpretation"] = "Ritme tidak jelas — mungkin ucapan terlalu datar."

    return analysis


## Test Fungsi

In [14]:
analyze_nonverbal(saved_result)

{'tempo_interpretation': 'Cepat — kemungkinan gugup atau terlalu bersemangat.',
 'pause_interpretation': 'Banyak jeda — bisa berarti gugup atau belum siap.',
 'rhythm_interpretation': 'Ritme bicara cukup konsisten.'}